# 📚 Catálogo de Dados - Camada Silver

## Tabela: `cripto`

### Descrição:
Tabela Silver que consolida os dados históricos e intradiários de criptoativos oriundos da camada Bronze. Essa tabela unifica os registros dos ativos financeiros, possibilitando análises integradas e continuidade das atualizações de forma incremental. A tabela foi criada para suportar futuras análises preditivas e servir como base para projetos de ciência de dados.

### Schema:
| Coluna           | Tipo        | Descrição                                                                                  | Domínio / Valores Esperados                  |
|------------------|-------------|--------------------------------------------------------------------------------------------|----------------------------------------------|
| asset_name       | string      | Nome do ativo financeiro. Origem: `metadata.asset_name` ou valor ajustado a partir de `ticker` (Histórico). | Ex: "bitcoin", "ethereum"                    |
| timestamp        | timestamp   | Data/hora da cotação efetiva do ativo.                                                    | Formato: YYYY-MM-DD HH:MM:SS; período histórico e intradiário. |
| price_usd        | double      | Preço do ativo em dólares no momento da cotação.                                           | Valor ≥ 0                                    |
| extraction_date  | timestamp   | Data/hora em que os dados foram extraídos da fonte original.                               | Data/hora da execução do pipeline            |
| source           | string      | Fonte de origem dos dados.                                                                 | Ex: "Yahoo Finance", "CoinGecko"             |
| time_interval    | string      | Intervalo de tempo referente aos dados intradiários. Para registros históricos, pode ser nulo. | Formato: "YYYY-MM-DD HH:MM:SS to HH:MM:SS"     |
| total_records    | int         | Número total de registros processados no arquivo de origem.                                | Valor inteiro ≥ 0                            |
| source_type      | string      | Indica a origem do registro na camada Bronze: "historico" ou "intradiario".                | Valor fixo: "historico" ou "intradiario"       |
| source_file      | string      | Nome do arquivo de origem que gerou o registro.                                            | Caminho ou nome do arquivo (para rastreabilidade)|

---

## Linhagem dos Dados

- **Origem:**  
  - `bronze_historico`: Arquivos JSON históricos em `landing/historico/{ativo}.json`.  
  - `bronze_intradiario`: Arquivos JSON diários em `landing/cripto/{ativo}/*.json`.

- **Transformação:**  
  - Os registros dos arquivos das camadas Bronze são lidos de forma incremental via Structured Streaming com Change Data Feed (CDF) e inseridos (append) na camada Silver.  
  - Os dados históricos e intradiários são unificados em uma única entidade: `ativo_financeiro`.

- **Destino:**  
  - Tabela Silver `ativo_financeiro` salva em formato Delta, garantindo time travel, otimizações com Liquid Clusters e integração com o Unity Catalog para auditabilidade e rastreamento da linhagem.

---

## Notas
- A tabela Silver está preparada para receber novos ativos de forma dinâmica. Caso um ativo novo seja detectado nos dados da camada Bronze, o pipeline captura todo o histórico desse ativo e passa a processá-lo de forma incremental.
- Esse catálogo faz parte de uma estratégia Lakehouse que visa unificar os dados de forma robusta para análises detalhadas e futuras aplicações de ciência de dados.

